## Домашнее задание 6

В данном домашнем задании Вам предстоит реализовать автоматическое исправление опечаток в запросах пользователей. 

### 1. Датасет
Для оценки качества алгоритма исправления опечаток, Вам предоставляется файл `queries.tsv.gz`. В каждой строке файла записаны два запроса – исходный и исправленный. Для простоты, оба запроса будут иметь одинаковое количество слов и отличаться незначительно. Зачастую исходный и исправленный запрос совпадают, что означает что исправлять такой запрос не требуется.

In [1]:
from typing import List, Tuple, Generator, Callable

Query = str
Sentence = str
Filename = str
Word = str
Queries = List[Tuple[Query, Query]]

In [2]:
from termcolor import colored
import difflib
import pickle

def diff_queries(original: Query, fixed: Query) -> Query:
    result = ''
    for pos, d in enumerate(difflib.ndiff(original, fixed)):
        if d[0] == '+':
            result += colored(d[2], 'green')
        elif d[0] == '-':
            result += colored(d[2], 'red')
        else:
            result += d[2]
    return result

print(diff_queries("lake compond the park", "lake compound the park"))
print(diff_queries("traditional chothes", "traditional clothes"))
print(diff_queries("jack sparrow", "captain jack sparrow"))

lake compound the park
traditional chlothes
captain jack sparrow


In [3]:
import gzip

def load_queries(fn: Filename) -> Queries:
    result = []
    with gzip.open(fn, 'rt', encoding='utf8') as inp:
        for line in inp:
            original, fixed = line.rstrip('\n').split('\t')
            result.append((original, fixed))
    return result

queries = load_queries("./queries.tsv.gz")
print(f'Loaded {len(queries)} queries\n')
for original, fixed in queries[10:20]:
    print(diff_queries(original, fixed))

Loaded 102436 queries

embearirassing red carpet moments
grants for rural areas florirda
the home hdepot merchandising
delaware motorcycle inspection requirements
highland park hospital gastric biypass surgery
grand theift auto
windward community college
my credit reports
strack intermediate school
mongol empire political system


In [4]:
queries_sample = [
    ("grand theift auto", "grand theft auto"),
    ("belarus longitude and latitdue", "belarus longitude and latitude"),
    ("search for poeoms", "search for poems"),
    ("large guacolmoi dip restaurtant price", "large guacamole dip restaurant price"),
    ("texas chainsaw mascurer", "texas chainsaw massacre"),
    ("royal trump subtitle", "royal tramp subtitle"),
    ("florida fiberglass polls", "florida fiberglass pools"),
    ("how to make a calender", "how to make a calendar"),
    ("university of south caroline", "university of south carolina"),
    ("maureen mcdonald in virginia", "maureen mcdonnell in virginia"),
]

Для составления словаря и обучения языковых моделей Вам предоставляется небольшой корпус текста, неслучайная выборка из большой английской википедии в файле `train.bz2`. Этот файл содержит примерно 5 млн строк или 80 млн слов. Каждая строка – одно предложение без знаков препинания.
Использование других словарей и корпусов запрещено.

In [5]:
import bz2
from tqdm import tqdm

def read_huge_corpus(fn: Filename) -> Generator[Sentence, None, None]:
    with bz2.open(fn, 'rt', encoding='utf8') as inp:
        for line in tqdm(inp):
            yield line.rstrip('\n')

for li, line in enumerate(read_huge_corpus("./train.bz2")):
    print(li, line)
    if li == 10:
        break

10it [00:00, 326.16it/s]

0 gol neshin
1 mitochondrial dna depletion syndrome mds or mdds is any of a group of autosomal recessive disorders that cause a significant drop in mitochondrial dna in affected tissues
2 following the relegation of sc freiburg in 2005 he was on the verge of signing for metalurg donetsk but instead he accepted a contract with vfl wolfsburg
3 the first issue for geometers is what kind of geometry is adequate for a novel situation
4 cedar grove was formerly a stage and freight stop
5 regular bus service runs from bhubaneswar to niali which is away
6 later they were also known for the cream wafer biscuits
7 strabomantis cornutus
8 gtk+ scene graph kit gsk was initially released as part of gtk+ 3.90 in march 2017 and is meant for gtk-based applications that wish to replace clutter for their ui
9 the match took place on 10 april 1906 at the hipódromo madrid
10 the brothers came from fresno california


### 2. Поиск близких слов
Требуется научится быстро находить список из сотни слов, которые незначительно отличаются от заданного слова.

Не стоит перебирать все слова словаря – займёт слишком много времени.

Для ускорения перебора предлагается создать триграммный индекс – для каждой буквенной триграммы храним список слов, в которых она есть. Тогда для поиска похожих на данное слово найдем слова большим количеством совпадающих триграмм. 

Совет 1: стоит сделать отельный индекс для каждой длинны слова и использовать только те индексы, в которых лежат слова близкие по длине к исходному.

Совет 2: для выделения триграмм стоит обрамить слово спецсимволом, чтобы триграммы на концах слова отличались от оных в середине.

Любые другие алгоритмы, улучшающие качество за разумное время (хождение по бору с ошибками, перебор ошибок) – не возбраняются.

Не побрезгуйте кешировать результат работы этого алгоритма, чтобы дальнейшая работа протекала быстрее.

In [18]:
id2word = {}
id2count = {}
word2id = {}

last_id = 0

for line in read_huge_corpus("./train.bz2"):
    for word in line.split():
        if word not in word2id:
            id2word[last_id] = word
            word2id[word] = last_id
            last_id += 1
        else:
            id_ = word2id[word]
            cur_count = id2count.get(id_, 1)
            id2count[id_] = cur_count + 1

4717753it [01:17, 61144.88it/s]


In [7]:
with open('id2word.p', 'rb') as f:
    id2word = pickle.load(f)

In [6]:
corpus = []

for line in read_huge_corpus("./train.bz2"):
    corpus += line.split()

4717753it [00:39, 120077.20it/s]


In [7]:
from collections import Counter

c = Counter([k for k in corpus])
c = {k:v for k, v in c.items() if v > 2}

In [8]:
from nltk import ngrams

def frame_word(v):
    return f"<<{v}>>"

def build_indexes(c, n=3):
    indexes = {}
    for v in tqdm(c.keys()):
        length_index = indexes.get(len(v), {})
        for ngram in ngrams(frame_word(v), n):
            str_ngram = ''.join(ngram)
            ngram_set = length_index.get(str_ngram, [])
            ngram_set.append(v)
            length_index[str_ngram] = ngram_set
        indexes[len(v)] = length_index
    
    return indexes

In [9]:
my_beautiful_indexes = build_indexes(c)

100%|██████████| 435212/435212 [00:06<00:00, 72368.74it/s] 


In [10]:
with open("id2count.p", 'wb') as f:
    pickle.dump(id2count, f)
with open("id2word.p", 'wb') as f:
    pickle.dump(id2word, f)
with open("word2id.p", 'wb') as f:
    pickle.dump(word2id, f)

Чтобы оценить качество полученного алгоритма, используйте запросы из `queries.tsv.gz`. Отберите только отличающиеся слова в исправленном и исходном запросах. Проверьте, что для слова в исходном запросе, исправленное слово будет в списке ближайших выданном вашим алгоритмом. Если это выполняется для всех или почти всех пар – успех. 

In [10]:
def extract_different_words(queries: Queries) -> List[Tuple[Word, Word]]:
    words_to_fix = []
    for original, fixed in queries:
        if original != fixed:
            for word_orig, word_fixed in zip(original.split(), fixed.split()):
                if word_orig != word_fixed:
                    words_to_fix.append((word_orig, word_fixed))
    return words_to_fix
                    
words_to_fix = extract_different_words(queries)
print(f'Found {len(words_to_fix)} words to fix')
for original, fixed in words_to_fix[:10]:
    print(diff_queries(original, fixed))

Found 53495 words to fix
chlothes
cataloges
compound
barnes
embearirassing
florirda
hdepot
inspection
biypass
theift


In [11]:
from collections import Counter
from nltk.util import ngrams

def find_similar_words(word: Word, n=3) -> List[Word]:
    length = len(word)
    word = frame_word(word)
    ngrms = list(ngrams(word, n))
    uniq_ngrms = set(ngrms)

    len_shifts = 2
    c = Counter({})
    for i in range(length - len_shifts, length + len_shifts + 1):
        if i > 0:
            index = my_beautiful_indexes.get(i, {})
            for ngram in ngrms:
                c += Counter(index.get(''.join(ngram), []))
                
    arr = [(w, count) for w, count in c.items() if count > 2]
                   
    sorted_arr = sorted(arr, key=lambda x: -x[1])
    return [w for w, _ in sorted_arr]

In [12]:
for original, fixed in words_to_fix[:5]:
    similar = find_similar_words(original)[:5]
   
    print(original,'- ok' if fixed in similar else '- fail')
    for word in similar[:5]:
        print(' ', word)
    print()

chothes - ok
  clothes
  chores
  chomes
  choses
  chokes

cataloges - ok
  catalogues
  catalogers
  catalogs
  cataloged
  catalogus

compond - ok
  compound
  composed
  component
  composted
  comarmond

barns - ok
  barns
  barnens
  barnes
  barons
  barnas

emberissing - ok
  embossing
  embarrassing
  embedding
  embezzling
  remembering



In [457]:
def check_find_similar_words(words_to_fix: List[Tuple[Word, Word]], 
                             find_similar_words: Callable[[Word], List[Word]], 
                             debug: bool):
    wrong, total = 0, 0
    progress = tqdm(words_to_fix)
    debug_output = 0
    for word_orig, word_fixed in progress:
        similar = find_similar_words(word_orig)
        if word_fixed not in similar:
            wrong += 1
            if debug:
                print(word_orig, word_fixed)
                debug_output += 1
                if debuge_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_find_similar_words(words_to_fix, find_similar_words, debug=False)

Wrong: 4275 - 7.99%: 100%|██████████| 53495/53495 [50:10<00:00, 17.77it/s]   


## 3. Языковая модель
Языковая модель – модель, которая по тексту оценивает вероятность того, что он мог появиться в языке. 

Постройте простую n-грамную языковую модель с использованием корпуса текстов `train.bz2`. Для этого рассчитайте количество вхождений каждой n-граммы в корпус текста. Если взять n=2, то размера оперативной памяти вашего компьютера должно будет хватить.

Воспользуйтесь каким-нибудь методом сглаживания, чтобы не получать нулевую вероятность для неизвестных n-грамм. Также, чтобы вероятности слов, которых нет в словаре, были отличны от нуля, можно примешать побуквенную m-граммную модель.

Совет N: если количество оперативной памяти прижмёт, можно хранить строки в виде байт – один раскодированный символ занимает больше памяти чем один байт, при этом для английского текста почти всегда один символ кодируется одним байтом.

In [13]:
def get_unigrams(sentences):
    unigrams={}
    for sentence in tqdm(sentences):
        tokens=sentence.split()
        for k in tokens:
            if k in c:
                if k in unigrams:
                    unigrams[k]+=1
                else:
                    unigrams[k]=1
    return unigrams

def get_bigrams(sentences):
    bigrams={}
    for sentence in tqdm(sentences):
        tokens=sentence.split()
        i=0
        length=len(tokens)-1
        while i<length:
            if tokens[i] not in bigrams:
                bigrams[tokens[i]]={}
            if tokens[i+1] not in bigrams[tokens[i]]:
                bigrams[tokens[i]][tokens[i+1]]=1
            else:
                bigrams[tokens[i]][tokens[i+1]]+=1
            i += 1
    return bigrams

In [14]:
sentences = [l for l in read_huge_corpus("./train.bz2")]

4717753it [00:26, 176770.01it/s]


In [15]:
unigrams = get_unigrams(sentences)

100%|██████████| 4717753/4717753 [00:40<00:00, 117517.67it/s]


In [16]:
bigrams = get_bigrams(sentences)

100%|██████████| 4717753/4717753 [01:29<00:00, 52781.97it/s]


In [82]:
dump = True

if dump:
    with open("bigrams.p", 'wb') as f:
        pickle.dump(bigrams, f)
    with open("unigrams.p", 'wb') as f:
        pickle.dump(unigrams, f)
else:
    with open('bigrams.p', 'rb') as f:
        bigrams = pickle.load(f)
    with open('unigrams.p', 'rb') as f:
        unigrams = pickle.load(f)

In [17]:
import math

def get_probability_1(sentence: Query) -> float:
    sentence = frame_query(sentence).split()
    score = 0.0
    for i in range(len(sentence) - 1):
        bigrams_count = bigrams.get(sentence[i], {}).get(sentence[i + 1], 0)
        if bigrams_count > 0:
            score += math.log(bigrams_count)
            score -= math.log(unigrams.get(sentence[i], 1))
        else:
            score += (math.log(unigrams.get(sentence[i + 1], 1) + 1) + math.log(0.4))
            score -= math.log(len(unigrams))
    return -1. / score


def get_probability_2(sentence: Query) -> float:
    sentence = sentence.split()
    score = unigrams.get(sentence[0], 0) / W
    lam = 0.79
    for i in range(len(sentence) - 2):
        bigrams_count = bigrams.get(sentence[i], {}).get(sentence[i + 1], 0)
        p2g = bigrams_count / unigrams.get(sentence[i], 1)
        #print(bigrams_count, unigrams.get(sentence[i], 1))

        pw = unigrams.get(sentence[i], 0) / W
        score *= (p2g * lam + (1 - lam) * pw)
        
    return score
        
    
    
def get_probability_3(sentence: Query) -> float:
    l = [0.6, 0.3, 0.1]
    sentence = frame_query(sentence).split()
    score = 0.
    for i in range(len(sentence) - 2):
        trigrams_count = trigrams.get(sentence[i], {}).get(sentence[i + 1], {}).get(sentence[i+2], 0) / tri_W
        bigrams_count = bigrams.get(sentence[i + 1], {}).get(sentence[i + 2], 0) / bi_W
        unigrams_count = unigrams.get(sentence[i + 2], 0) / W
        #print(sentence[i], sentence[i + 1], sentence[i+2])
        cur_score = trigrams_count * l[0] + bigrams_count * l[1] + unigrams_count * l[2]
        #print(cur_score)#, trigrams_count, bigrams_count, unigrams_count)
        score = cur_score
        
    return score


def get_probability(sentence: Query) -> float:
    sentence = sentence.split()
    score = 0.
    for i in range(len(sentence) - 1):
        bigrams_count = bigrams.get(sentence[i], {}).get(sentence[i + 1], 0)
        if bigrams_count > 0:
            score += 1
        else:
            score -= 1
            for w in [sentence[i], sentence[i + 1]]:
                if w not in c:
                    score -= 1
    return score


In [19]:
for original, fixed in queries_sample:
    p_original = get_probability(original)
    p_fixed = get_probability(fixed)
    verdict = '[ok]  ' if p_fixed > p_original else '[fail]'
    sign = '< ' if p_fixed > p_original else '>='
    print(f'{verdict} {original:>40s} {p_original:5.2f}  {sign} {p_fixed:5.2f} {fixed}')


[ok]                          grand theift auto -4.00  <   2.00 grand theft auto
[ok]             belarus longitude and latitdue -2.00  <   1.00 belarus longitude and latitude
[ok]                          search for poeoms -1.00  <   2.00 search for poems
[ok]      large guacolmoi dip restaurtant price -8.00  <  -2.00 large guacamole dip restaurant price
[ok]                    texas chainsaw mascurer -1.00  <   2.00 texas chainsaw massacre
[ok]                       royal trump subtitle -2.00  <   0.00 royal tramp subtitle
[fail]                 florida fiberglass polls -2.00  >= -2.00 florida fiberglass pools
[ok]                     how to make a calender  1.00  <   4.00 how to make a calendar
[ok]               university of south caroline  1.00  <   3.00 university of south carolina
[fail]             maureen mcdonald in virginia  3.00  >=  1.00 maureen mcdonnell in virginia


Чтобы оценить качество полученной модели, используйте запросы из `queries.tsv.gz`. Сравните вероятность, которую выдает ваша модель для исходных и исправленных запросов. Хорошая модель выдаёт исправленному запросу большую вероятность. 

In [46]:
def check_language_model(queries: Queries, get_probability: Callable[[Query], float], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        if original == fixed:
            continue
        p_original = get_probability(original)
        p_fixed = get_probability(fixed)
        if p_fixed <= p_original:
            wrong += 1
            if debug:
                print(original, p_original)
                print(fixed, p_fixed)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_language_model(queries, get_probability, debug=False)

Wrong: 4741 - 9.23%: 100%|██████████| 102436/102436 [00:44<00:00, 2298.68it/s]


Советую сохранить полученную модель на диск – а случае чего, чтение статистик с диска, может быть быстрее расчёта оных с нуля.

### 4. Модель ошибок
Модель ошибок – модель которая по исходному и исправленному запросу оценивает вероятность того, что такая ошибка могла быть допущена.

Рассчитайте простую модель ошибок на основе расстояния Дамерау-Левенштейна, то есть модифицированного Левенштейна, который считает перестановку соседних букв за одну ошибку.

In [20]:
import math
from math import log2

def lev(original: Word, fixed: Word) -> int:

    if not original:
        return len(fixed)
    if not fixed:
        return len(original)
    
    lenstr1 = len(original)
    lenstr2 = len(fixed)
    
    d = {}
    for i in range(-1, lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1, lenstr2+1):
        d[(-1,j)] = j+1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if original[i] == fixed[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1,
                           d[(i,j-1)] + 1,
                           d[(i-1,j-1)] + cost,
                          )
            if i and j and original[i] == fixed[j - 1] and original[i - 1] == fixed[j]:
                d[(i,j)] = min (d[(i,j)], d[i - 2,j - 2] + cost)
    return d[lenstr1-1,lenstr2-1]
    

def get_error_probability(original: Query, fixed: Query, a=1.5) -> float:
    l = lev(original, fixed)
    if l == 0:
        return 1.0
    else:
        return a ** -l

for original, fixed in queries_sample:
    p_error = get_error_probability(original, fixed)
    print(f'{original:>40s} | {p_error:5.2f} | {fixed}')

                       grand theift auto |  0.67 | grand theft auto
          belarus longitude and latitdue |  0.67 | belarus longitude and latitude
                       search for poeoms |  0.67 | search for poems
   large guacolmoi dip restaurtant price |  0.13 | large guacamole dip restaurant price
                 texas chainsaw mascurer |  0.20 | texas chainsaw massacre
                    royal trump subtitle |  0.67 | royal tramp subtitle
                florida fiberglass polls |  0.67 | florida fiberglass pools
                  how to make a calender |  0.67 | how to make a calendar
            university of south caroline |  0.67 | university of south carolina
            maureen mcdonald in virginia |  0.30 | maureen mcdonnell in virginia


## 5. Олтугеза
Объедините результат работы предыдущих пунктов в единый алгоритм исправления опечатки для запроса.

Примерный план:
1.	Для слов запроса генерируем список ближайших слов-кандидатов (для всех, даже словарных слов).
2.	Собираем список кандидатов-запросов (эвристически, чтобы не сделать экспоненциальное время выполнения)
3.	Для каждого кандидата считаем итоговый объединенный score на основе языковой модели и модели ошибок для данного кандидата (не обязательно сумма или произведение, можно объединение любой сложности).
4.	Выдаём гипотезу с наибольшим score.
5.	???
6.	Profit

In [43]:
import itertools


def correct0(query: Query) -> Query:
    similar_words = []
    splitted = query.split()
    for word in splitted:
        similars = find_similar_words(word)[:4]
        similar_words.append(similars)
    result = []
    for candidate in itertools.product(*similar_words):
        q = " ".join(candidate)
        score = get_error_probability(query, q) ** get_probability(q)
        result.append((score, q))
    
    return sorted(result, key=lambda x: x[0])[0][1]


def find_filtered_similar_words(word):
    similars = find_similar_words(word)[:2]
    words = [(get_error_probability(word, w), w) for w in similars]
    arr = [w for _, w in sorted(words, key=lambda x: -x[0])]

    return [word] if len(arr) == 0 else arr


def correct(query: Query) -> Query:
    similar_words = []
    splitted = query.split()
    for word in splitted:
        similars = find_filtered_similar_words(word)
        similar_words.append(similars)
    
    result = []
    for candidate in itertools.product(*similar_words):
        q = " ".join(candidate)
        result.append((-get_probability(q), q))
    return sorted(result, key=lambda x: x[0])[0][1]

In [181]:
for original, fixed in queries_sample:
    predict = correct(original)
    verdict = '[ok]  ' if predict == fixed else '[fail]'
    sign = '==' if predict == fixed else '!='
    print(f'{verdict} {predict:>40s} {sign} {fixed}')

[ok]                           grand theft auto == grand theft auto
[ok]             belarus longitude and latitude == belarus longitude and latitude
[ok]                           search for poems == search for poems
[fail] lafarge guacotecti dilip resistant prince != large guacamole dip restaurant price
[fail]                    texas chainsaw maurer != texas chainsaw massacre
[fail]                  royall trumps subtitled != royal tramp subtitle
[fail]                floridana fibreglass pols != florida fiberglass pools
[fail]                 how to make a calenderer != how to make a calendar
[fail]             university of south caroline != university of south carolina
[fail]             maureen mcdonald in virginia != maureen mcdonnell in virginia


Итоговое качество меряем на примерах из `queries.tsv.gz`.

Для отладки проблем с качеством имеет смысл научится понимать на каком этапе теряется правильная гипотеза для каждого примера. Например, если правильное исправление есть в списке кандидатов (п. 2), но не выбирается как лучшая – стоит крутить языковую модель, модель ошибок и их объединение.

In [26]:
def check_corrector(queries: Queries, correct: Callable[[Query], Query], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        predict = correct(original)
        if predict != fixed:
            wrong += 1
            if debug:
                print(original)
                print(fixed)
                print(predict)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')

Ждать все запросы долго, поэтому я перемешал и потестил 10% запросов

In [50]:
import random
queries_shuffled = queries
random.shuffle(queries_shuffled)
check_corrector(queries_shuffled[:10000], correct, debug=False)

Wrong: 2628 - 26.28%: 100%|██████████| 10000/10000 [57:08<00:00,  2.92it/s] 
